In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [46]:
df = pd.concat([pd.read_csv("../../data/cvrp-instances-1.0/train/cvrp-0-pa-train.csv"), pd.read_csv("../../data/cvrp-instances-1.0/dev/cvrp-0-pa-test.csv")])

In [47]:
df.shape

(120, 157)

In [48]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

In [49]:
Y = df["target"].copy()
X = df.drop(["target"], axis=1)
    
X_train, X_test, y_train, y_test = train_test_split(X.values, Y.values.tolist(), test_size=0.2, random_state=42)

In [50]:
def get_cv_results(grid_search_object, n=5):
    cv_results = pd.DataFrame(grid_search_object.cv_results_).sort_values('rank_test_score').head(n)
    return cv_results

In [51]:
cv_10 = KFold(10, shuffle=True, random_state=42)

In [76]:
xgb = XGBRegressor(random_state=42)
params = {'n_estimators': [10, 20, 30, 40, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900],
            'max_depth': [10, 20],
            'min_child_weight': [5, 10],
            'learning_rate': [0.01, 0.02],
            'subsample': [0.75],
            'colsample_bytree': [0.2, 0.3],
            'alpha': [0, 1, 2],
            'lambda': [0, 1, 10],
            'gamma': [2, 5, 10]
            }
xgb_cv = RandomizedSearchCV(xgb, param_distributions=params, scoring='neg_root_mean_squared_error', n_iter=20, cv=cv_10, random_state=42, verbose=1, n_jobs=-1)
xgb_cv.fit(X_train, y_train)

get_cv_results(xgb_cv, 10)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_min_child_weight,param_max_depth,param_learning_rate,param_lambda,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
18,0.466783,0.026042,0.002063,0.000066,0.75,700,5,10,0.01,10,...,-9256.867318,-4211.444994,-5050.214847,-5393.571883,-10205.732457,-8186.341479,-6151.528556,-6685.599824,1841.493247,1
3,0.661910,0.074974,0.003167,0.002261,0.75,700,5,20,0.01,1,...,-9017.506743,-4418.538308,-6002.058508,-5218.555536,-9828.893987,-7835.449434,-6221.053920,-6690.856653,1619.868763,2
16,0.783914,0.079952,0.002382,0.000376,0.75,900,5,20,0.02,0,...,-9420.122244,-4553.596501,-6446.504194,-4909.602740,-9488.667393,-7234.661337,-6358.361002,-6752.015641,1564.199300,3
4,0.181104,0.038541,0.002096,0.000502,0.75,200,10,10,0.02,0,...,-9565.200160,-4039.590396,-5676.060949,-4997.665171,-9877.887754,-8494.217719,-5818.700718,-6768.190014,1865.464682,4
17,0.557069,0.019469,0.002289,0.000139,0.75,700,5,20,0.01,0,...,-9139.527833,-4459.507096,-6359.302834,-4966.901413,-9391.799644,-7817.221063,-6274.540259,-6777.986480,1545.490420,5
6,0.272765,0.024104,0.002104,0.000265,0.75,400,10,10,0.01,0,...,-9484.866748,-4031.667162,-5708.474114,-5324.108557,-10006.538275,-8497.897986,-5602.542230,-6778.634210,1886.759322,6
10,0.147951,0.010898,0.001936,0.000116,0.75,200,5,20,0.02,1,...,-9762.572955,-4135.053584,-5134.456238,-5362.820325,-10386.985647,-8422.502010,-5358.095311,-6791.690918,1990.168720,7
0,0.093794,0.010241,0.001863,0.000183,0.75,200,10,10,0.02,1,...,-9620.211541,-4303.814772,-5056.239187,-5371.147342,-10263.744788,-8584.399816,-5116.730514,-6798.330082,1990.469478,8
5,0.982474,0.081143,0.002907,0.000938,0.75,800,5,10,0.02,0,...,-9419.225224,-4412.004350,-6571.071472,-4930.922856,-9685.386068,-7278.224776,-6356.128230,-6820.590393,1602.460218,9
19,0.351865,0.036044,0.001877,0.000476,0.75,700,10,10,0.02,0,...,-9486.674105,-4615.225895,-6447.475948,-5584.721358,-9954.202976,-7732.010265,-7071.043871,-7027.079811,1578.783586,10


In [77]:
xgb_cv.best_params_

{'subsample': 0.75,
 'n_estimators': 700,
 'min_child_weight': 5,
 'max_depth': 10,
 'learning_rate': 0.01,
 'lambda': 10,
 'gamma': 5,
 'colsample_bytree': 0.3,
 'alpha': 2}

In [78]:
xgb = XGBRegressor(subsample=0.75, n_estimators=700, min_child_weight=5, max_depth=10, learning_rate=0.01, gamma=10, colsample_bytree=0.3, alpha=1, random_state=42, eval_metric="rmse")
xgb.fit(X_train, y_train)

XGBRegressor(alpha=1, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=10, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             n_estimators=700, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)

In [79]:
y_pred = xgb.predict(X_test)

In [83]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(y_test, y_pred)

0.06866218567398531